## Ordnance Survey Data Hub Tutorials - Example Applications

# Using the OS Names API with ipyleaflet

This is a simple Python/Jupyter Notebook application, demonstrating how the OS Names API can be used to populate selected 
point features within an interactive ipyleaflet (https://ipyleaflet.readthedocs.io/en/latest/) map.  Aside from using the OS Names API itself, the application also makes use of the OS Maps API, as a source of contextual background mapping.  In this example, we'll search for school names in Winchester, searching for primary schools within a supplied bounding box, along with the nearest secondary school.  Together, this will demonstrate the two primary query modes of this service, along with its ability to filter results.

The OS Names API, one of the many services available from the OS Data Hub, is a geographic directory containing basic information about identifiable places.  This service is free to use for all registered OS Data Hub users.



In [1]:
from datetime import datetime
from ipyleaflet import Map, WidgetControl, AwesomeIcon, Marker, MarkerCluster
from ipywidgets import Image
from urllib.parse import unquote, urlencode
from pyproj import Transformer, crs
import IPython
import requests

Use of the OS APIs requires a service key.  Sign up for yours at <a>https://osdatahub.os.uk/</a> and enter it below.

In [2]:
# OS Data Hub project API key
key = 'YOUR API KEY HERE'

The following cells set up a basic ipyleaflet map object, configured to use the OS Maps API.  It will be initially centred on Winchester (UK) city centre, with a zoom level of 12.  It will use the 'Light' style, which is one of the default styles offered by this service.  This is a good choice for general background mapping. 

In [3]:
# Function to create ipyleaflet tile layers
def createOSMapsTileLayer(layertype):

    params = urlencode({'key': key,
                        'service': 'WMTS',
                        'request': 'GetTile',
                        'version': '2.0.0',
                        'height': 256,
                        'width': 256,
                        'outputFormat': 'image/png',
                        'style': 'default',
                        'layer': layertype,
                        'tileMatrixSet': 'EPSG:3857',
                        'tileMatrix': '{z}',
                        'tileRow': '{y}',
                        'tileCol': '{x}'})

    # OS Data Hub base path - https://api.os.uk
    # OS Maps API WMTS end point path - /maps/raster/v1/wmts?
    url = unquote(f'https://api.os.uk/maps/raster/v1/wmts?{params}')

    tileLayer = {'url': url,
               'min_zoom': 7,
               'max_zoom': 20,
               'attribution': f'Contains OS data &copy; Crown copyright and database rights {datetime.now().year}'}

    return tileLayer

In [4]:
# Create custom ipyleaflet TileLayer for the OS Maps API WMTS resource
os_maps_api_light = createOSMapsTileLayer('Light_3857')

In [5]:
# Create ipyleaflet Map
centreOfMap = [51.063650 , -1.3197756]  # WGS84
m = Map(basemap=os_maps_api_light,
        center=centreOfMap,
        zoom=12)

In [6]:
# Add an image overlay, placing the OS logo in the bottom left corner.
oslogo_url = 'https://raw.githubusercontent.com/OrdnanceSurvey/os-api-branding/master/img/os-logo-maps.png'
oslogo_img = IPython.display.Image(oslogo_url, width = 300)
oslogoWidget = Image(value=oslogo_img.data, format='png', width=100, height=30)
widgetControl = WidgetControl(widget=oslogoWidget, position='bottomleft')
m.add_control(widgetControl)


The OS Names API only returns British National Grid (BNG) coordinates.  For this application, we need latitude/longitude values, so the following function, using the pyproj package (https://pyproj4.github.io/pyproj/stable/), undertakes a conversion between the two.

In [7]:
def convertBNGtoLatLon(eastings, northings):

    # Convert a BNG coordinate pair to lat/lon
    bng = crs.CRS('epsg:27700')
    wgs84 = crs.CRS('epsg:4326')

    transformer = Transformer.from_crs(bng, wgs84)
    wgs84_point = transformer.transform(eastings, northings)

    return wgs84_point


This function takes a list of feature data and converts them to icons that will then be added to an ipyleaflet layer and added to our map object.  The icons, themselves, use the Font-Awesome library, which can be referenced at https://fontawesome.com/v4.7.0/icons

In [8]:
def createNamesAPIAweIconLayer(simplifiedlist, iname, mcolour, icolour):
    # Because it's currently hard to style point markers when returned in a GeoJSON layer, this is a simple alternative that uses Awesome Icons instead.
    
    mymarkers = []

    for d in simplifiedlist:

        # This API does not return lat/long values, so convert from BNG.
        thisLoc = convertBNGtoLatLon(d["GEOMETRY_X"],d["GEOMETRY_Y"])

        thisIcon = AwesomeIcon(name=iname, marker_color=mcolour, icon_color=icolour,spin=False)  
        thisMarker = Marker(icon=thisIcon, location=(thisLoc[0],thisLoc[1]), opacity=1, visible=True)
        mymarkers.append(thisMarker)

    # Finally, add the results to the map as a complete marker cluster
    m.add_layer(MarkerCluster(markers=tuple(mymarkers)))

Data returned from the OS Names API is standard JSON (i.e. not GeoJSON).  It contains more content and complexity than we really need for this specific application.  We don't really need anything from it's header, so we can disregard that.  In addition, the main body contains a hierarchy that we don't really need, so we'll use the following function to flatten it down into a simple list of dictionaries.  To better understand what's going on here, it's worth visualising the returned JSON directly.

Once we've got data in this format, several options are available to us.  For example, we could easily import them into a DataFrame/GeoDataFrame, or can convert them into actual GeoJSON.  In this example, though, we can make good use of the data directly. 

In [9]:
# Simplify the returned JSON data
def simplifyJSONResponse(jsondata):

    try:
        resultslist = jsondata['results'].copy()  # get just the list of results, disregarding the header
    except:
        raise Exception("The returned JSON data does not appear to have a 'results' block.  This indicates an erroneous or null return.")

    simplifiedlist = []

    for i in range(len(resultslist)):    # For the Names API, each result is a one-entry dictionary, with a further dictionary nested within.
        outerDict = resultslist[i]

        if "GAZETTEER_ENTRY" in outerDict:  # All results from the should be of type "GAZETTEER_ENTRY"
            simplifiedlist.append(outerDict["GAZETTEER_ENTRY"])
        else:
            raise Exception("Unexpected data, as only 'GAZETTEER_ENTRY' blocks are expected for this API.  Check what's being returned.")

    return simplifiedlist

Build the two URLs we'll need for the OS Names API queries - a 'radius' type query for primary schools and a 'nearest' search for the secondary school.

In [10]:
# Get OS Names API JSON data
# We're requesting the result to be returned in WGS84, as that's the only CRC supported by geojson.  It should all work out in the end.

osnames_base_url = 'https://api.os.uk/search/names/v1/'  # base url for all OS Names queries

primary_params = 'find?query=Winchester&fq=BBOX:444135,126565,450393,132885&maxresults=50' # additional parameters for 'find' variant
secondary_params = 'nearest?point=447763,129588&radius=1000' # additional parameter for 'nearest' variant

primary_filter = '&fq=LOCAL_TYPE:Primary_Education' # filters are optional, but here we just want to choose primary schools...
secondary_filter = '&fq=LOCAL_TYPE:Secondary_Education' # ...or secondary schools.  See https://osdatahub.os.uk/docs/names/technicalSpecification for all options.

full_primary_url = osnames_base_url + primary_params + primary_filter + '&format=json&key=' + key 
full_secondary_url = osnames_base_url + secondary_params + secondary_filter + '&format=json&key=' + key 

# Print full URLs, for reference
print (full_primary_url)
print (full_secondary_url)



https://api.os.uk/search/names/v1/find?query=Winchester&fq=BBOX:444135,126565,450393,132885&maxresults=50&fq=LOCAL_TYPE:Primary_Education&format=json&key=rYQ4O27ZlRGAYUaZyMG3UuYd0Io9uZPS
https://api.os.uk/search/names/v1/nearest?point=447763,129588&radius=1000&fq=LOCAL_TYPE:Secondary_Education&format=json&key=rYQ4O27ZlRGAYUaZyMG3UuYd0Io9uZPS


Undertake the two OS Names API queries, process the results and add the resulting icon layers to the map.

In [11]:
try:
    primary_response = requests.get(full_primary_url)
    secondary_response = requests.get(full_secondary_url)
except:
    raise Exception("Unable to query and process Names API.")

# Convert results to JSON
try:
    primary_data = primary_response.json()
    secondary_data = secondary_response.json()
except:
    raise Exception("Unable to convert the returned payload to JSON.")

# We'll create and populate simplified feature lists...
simplifiedlist_primary = simplifyJSONResponse(primary_data)
simplifiedlist_secondary = simplifyJSONResponse(secondary_data)

# ...then use these to create new map layers
if simplifiedlist_primary:
    createNamesAPIAweIconLayer(simplifiedlist_primary, 'mortar-board', 'purple', 'black')

if simplifiedlist_secondary:
    createNamesAPIAweIconLayer(simplifiedlist_secondary, 'university', 'green', 'white')


Finally, display the completed interactive ipyleaflet map.

In [12]:
m

Map(center=[51.06365, -1.3197756], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…